In [8]:
from guess_number import guess_number, average_int_result
from numpy import mean, random
from math import log2

guess_number = average_int_result(guess_number)
for hi in [2**i for i in range(10,21)]:
    print(f"\n[1; 2**{int(log2(hi))}]")
    print(f"Average number of tries is {guess_number(1, hi)}")


[1; 2**10]
1  - 1019 
2  - 1957 
3  - 3947 
4  - 7892 -
5  - 15809--
6  - 31395---
7  - 62702------
8  - 124861------------
9  - 249273=========================
10 - 500245--------------------------------------------------
11 - 900  
Average number of tries is 9

[1; 2**11]
1  - 494  
2  - 981  
3  - 1976 
4  - 3969 
5  - 7721 -
6  - 15794--
7  - 31166---
8  - 62239------
9  - 125235-------------
10 - 250395=========================
11 - 499564--------------------------------------------------
12 - 466  
Average number of tries is 10

[1; 2**12]
1  - 253  
2  - 438  
3  - 1011 
4  - 1903 
5  - 3942 
6  - 7647 -
7  - 15611--
8  - 31267---
9  - 62385------
10 - 125291-------------
11 - 250209=========================
12 - 499815--------------------------------------------------
13 - 228  
Average number of tries is 11

[1; 2**13]
1  - 114  
2  - 261  
3  - 492  
4  - 976  
5  - 1942 
6  - 3944 
7  - 7720 -
8  - 15625--
9  - 31157---
10 - 62500------
11 - 124948------------
12 - 250350==